In [ ]:
import numpy as np
from scipy.integrate import odeint
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from sklearn import preprocessing
from tqdm import tqdm

# COOL – Three-Stage Reactor Cascade with Countercurrent Cooling

## Parameters

In [ ]:
N_reactors = 3

# Burst duration, s
BURST=3000     

# Frequency factor, 1/s
Z = 2000
# Activation energy, kJ/kmol 
E = 4.23e+4
# Gas constant, kJ/kmol K
R = 8.290

# Inlet feed concentration, kmol/m3
C0 = 0.600
Tc0 = 298

# Flow rate, m3/s
F0 = 2.5e-3    
# Coolant flow rate, m3/s
Fc0 = 2.0e-3
# Heat of reaction, kJ/kmol
Hr = -1.465e+5
# Volume of each reactor, m3
V = 0.8
# Volume of each jacket, m3
V_c = 0.100
# Density,kg/m3
rho = 1.0e+3
# Density of coolant, kg/m3
rho_c = 1000# Heat capacity, kJ/kg K
c_p = 0.565
# Heat capacity of coolant, kJ/kg K
c_pc = 4.187

# Tc for coolant flow rate
# Heat transfer coeff., kJ/m2 s K
U = 0.6
# Area for heat trarea for hnsfer, m2
A = 0.5

# Proportional eff FeedBack control of feed flow rate
KC = 0.00001
# Proportional eff FeedBack control of coolant flow rate
KC_C = 0.0001
# Setpoint of T[-1]
Tset = 330

In [ ]:
n_timestamps = 100000
Flag_noise = True
np.random.seed(2021)

## Generate inputs of system

In [ ]:
def gen_signal(stage, transition_stage, n_timestamps, LB, UB, distribution="Uniform", seed=2021):
    '''
    Generate stairstep signal with a linear transition
    '''
    np.random.seed(seed)
    F_stage = stage
    F_transition_stage = transition_stage
    F_comb_num = int(np.ceil(n_timestamps/ (stage + transition_stage)))
    
    if distribution == "Uniform":
        value_stage = np.random.uniform(LB, UB, size=(F_comb_num + 1,))
    elif distribution == "Normal":
        value_stage = np.random.normal(LB, UB, size=(F_comb_num + 1,))
        
    F_ = []
    for i in range(F_comb_num):
        F_.append(list(np.ones((F_stage, )) * value_stage[i]))
        F_.append(list(np.linspace(value_stage[i], value_stage[i + 1], F_transition_stage)))
    F_ = [item_ for item in F_ for item_ in item]
    return F_[:n_timestamps]      

In [ ]:
# C0
C0_1 = gen_signal(700, 300, int(n_timestamps / 3), 0.54, 0.64, seed=2021)
C0_2 = gen_signal(700, 300, int(n_timestamps / 3), 0.58, 0.68, seed=2021)
C0_3 = gen_signal(700, 300, n_timestamps - int(n_timestamps / 3) - int(n_timestamps / 3), 0.56, 0.68, seed=2021)
C0_ = C0_1 + C0_2 + C0_3
C0 = np.array(C0_ + [C0_[-1]]).astype(float)

# F
F_ = gen_signal(500, 30, n_timestamps,  1.8 * 1e-3, 2.9 * 1e-3, seed=2021)
F = np.array(F_ + [F_[-1]]).astype(float)

# Tset
Tset = 330 * np.ones(n_timestamps + 1,)

# Tc0
Tc0 = np.ones(n_timestamps + 1,) * Tc0

# Fc
Fc = np.ones(n_timestamps + 1,) * Fc0

T_noise_mean = 0
T_noise_std = 0.5

if Flag_noise:
    np.random.seed(2021)
    T0_noise = np.random.normal(303.5, 0.1, size=(n_timestamps + 1,)) + 0.8 * np.sin(np.arange(1, n_timestamps + 2) * 2 * np.pi / 720)
    Tc0_noise = np.random.normal(295, 0.1, size=(n_timestamps + 1,)) + 0.8 * np.sin(np.arange(1, n_timestamps + 2) * 2 * np.pi / 1440)
    
    C0_noise = (C0 + np.random.normal(0,0.003, n_timestamps + 1)).astype(float) + 0.005 * np.sin(np.arange(1, n_timestamps + 2) * 2 * np.pi / 480)
    Fc_noise = (Fc + np.random.normal(0,0.0001, n_timestamps + 1)).astype(float)
    F_noise = (F + np.random.normal(0,0.00002, n_timestamps + 1)).astype(float) + 0.00005 * np.sin(np.arange(1, n_timestamps + 2) * 2 * np.pi / 240)
else:
    T0_noise, Tc0_noise, C0_noise, Fc_noise, F_noise = 0.8 * np.sin(np.arange(1, n_timestamps + 2) * 2 * np.pi / 720), Tc0, C0, Fc, F

In [ ]:
fig = make_subplots(rows=4, cols=1, vertical_spacing=0.04, shared_xaxes=True)
fig.add_trace(go.Scatter(x=[i for i in range(n_timestamps)], y=T0_noise, line_color="blue", showlegend=False), row = 1, col = 1)
fig.add_trace(go.Scatter(x=[i for i in range(n_timestamps)], y=Tc0_noise, line_color="blue", showlegend=False), row = 2, col = 1)
fig.add_trace(go.Scatter(x=[i for i in range(n_timestamps)], y=C0_noise, line_color="blue", showlegend=False), row = 3, col = 1)
fig.add_trace(go.Scatter(x=[i for i in range(n_timestamps)], y=F_noise, line_color="blue", showlegend=False), row = 4, col = 1)
fig.update_layout(width=1300, height=800, plot_bgcolor='white'
                  , font=dict(size=15
                              , family="Times New Roman"
                             )
                 )
fig['layout']['xaxis4']['title']='<b>Time<b>'
fig['layout']['yaxis1']['title']= 'T_0_noisy'
fig['layout']['yaxis2']['title']= 'T_c0_noisy'
fig['layout']['yaxis3']['title']= 'C_0_noisy'
fig['layout']['yaxis4']['title']= 'F_noisy'

fig.update_xaxes(showline=True, linewidth=2, linecolor='black', gridcolor='black', mirror=True, ticks='outside')
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', gridcolor='black', mirror=True, ticks='outside')
fig.show()

## Simulation

In [ ]:
def DiffEquations_Cool(Vars, t, N_reactors, F, Fc, T0, Tc0, C0):
    '''
    First-principles model of the system
    '''
    C = Vars[:N_reactors]
    T = Vars[N_reactors: N_reactors * 2]
    Tc = Vars[N_reactors * 2: N_reactors * 3]

    k = Z * np.exp(-E / (R * T))

    # The first reactor
    dC0dt = (F * (C0 - C[0]) - V * k[0] * C[0]) / V
    dT0dt = (F * rho * c_p * (T0 - T[0]) - V * k[0] * C[0] * Hr - U * A * (T[0] - Tc[-1])) / (rho * c_p * V)
    
    # Middle Reactors
    dCmid_dt = (F * (C[0:-2] - C[1:-1]) - V * k[1:-1] * C[1:-1]) / V
    dTmid_dt = (F * rho * c_p * (T[0:-2] - T[1:-1]) - V * k[1:-1] * C[1:-1] * Hr - U * A * (T[1:-1] - Tc[-2:0:-1])) / (rho * c_p * V)
    
    # The last reactors
    dClast_dt = (F * (C[-2] - C[-1]) - V * k[-1] * C[-1]) / V
    dTlast_dt = (F * rho * c_p * (T[-2] - T[-1]) - V * k[-1] * C[-1] * Hr - U * A * (T[-1] - Tc[0])) / (rho * c_p * V)
    
    # Tc in inverse order
    dTc0dt = (Fc * rho_c * c_pc * (Tc0 - Tc[0]) + U * A * (T[-1] - Tc[0])) / (rho_c * c_pc * V_c)
    dTcmid_dt = (Fc * rho_c * c_pc * (Tc[0:-2] - Tc[1:-1]) + U * A * (T[-2:0:-1] - Tc[1:-1])) / (rho_c * c_pc * V_c)
    dTclast_dt = (Fc * rho_c * c_pc * (Tc[-2] - Tc[-1]) + U * A * (T[0] - Tc[-1])) / (rho_c * c_pc * V_c)

    dydt = np.concatenate([np.array([dC0dt]), dCmid_dt, np.array([dClast_dt]), np.array([dT0dt]), dTmid_dt, np.array([dTlast_dt])
    , np.array([dTc0dt]), dTcmid_dt, np.array([dTclast_dt])])
    return dydt

In [ ]:
t = np.linspace(0, 60, 60 + 1)
initial_values = np.array([0.6,0.6,0.6,300.,310.,330.,298.,298.,298.])
Flag_Tc_F=True
Flag_Tc_C=False
arr_y = np.zeros((n_timestamps + 1, 9))
arr_y[0, :] = [0.6,0.6,0.6,300.,310.,340.,298.,298.,298.]
list_F = [F0]
list_Fc = [Fc0]
list_C0 = [C0[0]]

SE = 0
F__, Fc__, T0__, Tc0__, C0__ = F0, 2.0e-3, T0_noise[0], Tc0[0], C0[0]
for i in tqdm(range(n_timestamps)):
    y = odeint(DiffEquations_Cool, initial_values, t, args=(N_reactors, F__, Fc__, T0__, Tc0__, C0__))
    arr_y[i + 1, :] = y[-1, :]
    T__ = y[-1, 3:6]
    
    initial_values = y[-1, :]
    T0__, Tc0__, C0__ = T0_noise[i + 1], Tc0_noise[i + 1], C0_noise[i + 1]
    
    # A simple controller
    Tset_ = Tset[i + 1]
    SE += Tset_ - T__[-1]
    F__ = F_noise[i + 1]
    Fc__ = Fc0 - (0.00005 * (Tset_ - T__[-1]) +  0.0000005 * SE)   

    # Add limit to flowrate of feed flow and coolant flow
    F__ = 0 if F__ < 0 else F__
    Fc__ = 0 if Fc__ < 0 else Fc__

    list_F.append(F__)
    list_Fc.append(Fc__)
    list_C0.append(C0__)

In [ ]:
t = np.linspace(0, n_timestamps, n_timestamps + 1)
fig = make_subplots(rows=5, cols=1, shared_xaxes=True, vertical_spacing=0.05
                    , subplot_titles=["Noise-free Concentration of Reactors", "Noise-free Reactors Temperatures"
                                        , "Noise-free Coolwater temperatures", "F", "Fc", "C0"])
for i in range(3):
    fig.add_trace(go.Scatter(x = t, y = arr_y[:, i], name=f"C[{i+1}]"), row=1, col=1)
for i in range(3):
    fig.add_trace(go.Scatter(x = t, y = arr_y[:, i + 3], name=f"T[{i+1}]"), row=2, col=1)    
for i in range(3):
    fig.add_trace(go.Scatter(x = t, y = arr_y[:, i + 6], name=f"Tc[{i+1}]"), row=3, col=1)    
fig.add_trace(go.Scatter(x=t, y=list_F), row = 4, col = 1)
fig.add_trace(go.Scatter(x=t, y=list_Fc), row = 5, col = 1)    
fig.add_trace(go.Scatter(x=t, y=list_C0), row = 1, col = 1)  
fig.add_trace(go.Scatter(x=t, y=Tset), row = 2, col = 1)  
fig.update_layout(width=1300, height=800, plot_bgcolor='white'
                  , font=dict(size=15
                              , family="Times New Roman"
                             )
                 )
fig.update_xaxes(showline=True, linewidth=2, linecolor='black', gridcolor='black', mirror=True, ticks='outside')
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', gridcolor='black', mirror=True, ticks='outside')
fig.show()

## Add noise to ***C***,***T***,***Tc***

In [ ]:
if Flag_noise:
    y_noise = arr_y.copy()
    y_noise[:, :3] = y_noise[:, :3] + np.random.normal(0,0.001,(n_timestamps + 1, 3))
    y_noise[:, 3:6] = y_noise[:, 3:6] + np.random.normal(0,0.5,(n_timestamps + 1, 3))
    y_noise[:, 6:9] = y_noise[:, 6:9] + np.random.normal(0,0.5,(n_timestamps + 1, 3))
else:
    y_noise = arr_y.copy()
    
arr_F = np.array(list_F)
arr_Fc = np.array(list_Fc)    

In [ ]:
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.04)

list_color = ["blue", "red", "green"]
list_C = [r"$C_{1}$",r"$C_{2}$",r"$C_{3}$"]
list_T = [r"$T_{1}$",r"$T_{2}$",r"$T_{3}$"]
list_Tc = [r"$T_{c1}$",r"$T_{c2}$",r"$T_{c3}$"]

for i in range(3):
    fig.add_trace(go.Scatter(x = t, y = y_noise[:, i], name=list_C[i], line_color=list_color[i], legendgroup="1"
                            ), row=1, col=1)
for i in range(3):
    fig.add_trace(go.Scatter(x = t, y = y_noise[:, i + 3], name=list_T[i], line_color=list_color[i], legendgroup="2"
                            ), row=2, col=1)    
for i in range(3):
    fig.add_trace(go.Scatter(x = t, y = y_noise[:, i + 6], name=list_Tc[i], line_color=list_color[i], legendgroup="3"
                            ), row=3, col=1)    
fig.update_layout(width=1300, height=800, plot_bgcolor='white'
                  , font=dict(size=15
                              , family="Times New Roman"
                             )
                 )
fig['layout']['xaxis3']['title']= '<b>Time<b>'
fig['layout']['yaxis1']['title']= '<b>Noisy Concentrations<b>'
fig['layout']['yaxis2']['title']= '<b>Noisy reactor<br>temperatures<b>'
fig['layout']['yaxis3']['title']= '<b>Noisy cooling water<br>temperature<b>'

fig.update_layout(legend=dict(yanchor="top", xanchor="left",y=1,x=1.001,tracegroupgap=160,bgcolor="white"))

fig.update_xaxes(showline=True, linewidth=2, linecolor='black', gridcolor='gray', mirror=True, ticks='outside')
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', gridcolor='gray', mirror=True, ticks='outside')
# fig.write_image("HTML/20220310_30min_1minTesting/Gen_signal/Output.png")
fig.show()

# Construct input and output dataset

In [ ]:
idx_start = 200
t_sel = [i for i in range(len(t) - idx_start)]
y_sel = y_noise[idx_start:]
T0_noise_sel = T0_noise[idx_start:]
Tc0_noise_sel = Tc0_noise[idx_start:]
F_noise_sel = F_noise[idx_start:]
Fc_noise_sel = Fc_noise[idx_start:]
C0_noise_sel = C0_noise[idx_start:]
Tset_sel = Tset[idx_start:]

In [ ]:
df_input = pd.DataFrame(index = t_sel, columns = ["F", "Fc", "T1", "T2", "T3", "Tc1", "Tc2", "Tc3", "T0", "Tc0", "C0", "Tset"])
df_input["F"] = F_noise_sel.reshape(-1, 1)
df_input["Fc"] = Fc_noise_sel.reshape(-1, 1)
df_input["T0"] = T0_noise_sel.reshape(-1, 1)
df_input["Tc0"] = Tc0_noise_sel.reshape(-1, 1)
df_input["C0"] = C0_noise_sel.reshape(-1, 1)
df_input[["T1", "T2", "T3"]] = y_sel[:, 3:6]
df_input[["Tc1", "Tc2", "Tc3"]] = y_sel[:, 6:9]
df_input["Tset"] = Tset_sel.reshape(-1, 1)

In [ ]:
# Get a C for every hour
C_rate = 60
idx_C = [i * C_rate for i in range(1, int(len(t_sel) / C_rate))]
df_output = pd.DataFrame(index = idx_C, columns = ["C1", "C2", "C3"])
df_output[["C1", "C2", "C3"]] = y_sel[idx_C, 0:3]

In [ ]:
# Normalization
scaler_input = preprocessing.MinMaxScaler().fit(df_input)
df_input_scale = pd.DataFrame(columns=df_input.columns, index=df_input.index)
df_input_scale.iloc[:, :] = scaler_input.transform(df_input)

scaler_output = preprocessing.MinMaxScaler().fit(df_output)
df_output_scale = pd.DataFrame(columns=df_output.columns, index=df_output.index)
df_output_scale.iloc[:, :] = scaler_output.transform(df_output)

In [ ]:
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.04)
list_color = ["blue", "red", "green"]
for i in range(3):
    fig.add_trace(go.Scatter(x = [i for i in range(df_output.shape[0])], y = df_output.iloc[:, i]
                             , line_color=list_color[i], legendgroup=f"{i+1}", name=list_C[i], showlegend=False
                            ), row=i+1, col=1)
    
fig['layout']['xaxis3']['title']= '<b>Time<b>'
fig['layout']['yaxis1']['title']= r'$\text {Sampled  } C_{1}$'
fig['layout']['yaxis2']['title']= r'$\text {Sampled  } C_{2}$'
fig['layout']['yaxis3']['title']= r'$\text {Sampled  } C_{3}$'

fig.update_layout(width=1300, height=800, plot_bgcolor='white'
                  , font=dict(size=15
                              , family="Times New Roman"
                             )
                 )
fig.update_layout(legend=dict(yanchor="top", xanchor="left",y=1,x=1.001,tracegroupgap=200,bgcolor="white"))

fig.update_xaxes(showline=True, linewidth=2, linecolor='black', gridcolor='gray', mirror=True, ticks='outside')
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', gridcolor='gray', mirror=True, ticks='outside')
# fig.write_image("HTML/20220310_30min_1minTesting/Gen_signal/Sampled_Output.png")
fig.show()

## sample rate 1min 

In [ ]:
df_output_1min = pd.DataFrame(index = [i for i in range(y_sel.shape[0])], columns = ["C1", "C2", "C3"])
df_output_1min[["C1", "C2", "C3"]] = y_sel[:, 0:3]

In [ ]:
# scaler_output_1min = preprocessing.MinMaxScaler().fit(df_output_1min)
df_output_1min_scale = pd.DataFrame(columns=df_output_1min.columns, index=df_output_1min.index)
df_output_1min_scale.iloc[:, :] = scaler_output.transform(df_output_1min)

In [ ]:
df_data = pd.concat([df_input, df_output_1min], axis=1)
df_data.to_csv("dataset.csv")